# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
%load_ext dotenv
%dotenv 

In [2]:
import os
import sys

sys.path.append(os.getenv('SRC_DIR'))

In [3]:
import dask.dataframe as dd

C:\Users\ritup\AppData\Local\Temp\ipykernel_19484\676544213.py:1: DeprecationWarning: The current Dask DataFrame implementation is deprecated. 
In a future release, Dask DataFrame will use new implementation that
contains several improvements including a logical query planning.
The user-facing DataFrame API will remain unchanged.

The new implementation is already available and can be enabled by
installing the dask-expr library:

    $ pip install dask-expr

and turning the query planning option on:

    >>> import dask
    >>> dask.config.set({'dataframe.query-planning': True})
    >>> import dask.dataframe as dd

API documentation for the new implementation is available at
https://docs.dask.org/en/stable/dask-expr-api.html

Any feedback can be reported on the Dask issue tracker
https://github.com/dask/dask/issues 

  import dask.dataframe as dd


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
PRICE_DATA = os.getenv("PRICE_DATA")

In [5]:
from glob import glob
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/part.0.parquet"), recursive=True)
parquet_files

['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

In [6]:
df = dd.read_parquet(parquet_files)
df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
npartitions=11132,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32
,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [7]:
df.columns

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')

In [8]:
import numpy as np
dd_rets = (df.groupby('ticker', group_keys=False)
# .apply(
#     lambda x: x.assign(Close_lag = x['Close'].shift(1)))
.apply(
    lambda x: x.assign(Adj_Close_lag = lambda x: x['Adj Close'].shift(1)))
.assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1)
.assign(
    hi_lo_range = lambda x: x['High'] - x['Low'])
)

C:\Users\ritup\AppData\Local\Temp\ipykernel_19484\830134880.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_rets = (df.groupby('ticker', group_keys=False)


In [9]:
# dd_rets = (df.groupby('ticker', group_keys=False)
# .apply(
#     lambda x: x.assign(Close_lag = x['Close'].shift(1)))
# # .assign(
# #     Adj_Close_lag = lambda x: x['Adj Close'].shift(1))
# .assign(
#     returns = lambda x: x['Close']/x['Close_lag'] - 1)
# .assign(
#     hi_lo_range = lambda x: x['High'] - x['Low'])
# )

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [10]:
dd_rets = (dd_rets.groupby('ticker', group_keys=False)
.apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1)))
)

C:\Users\ritup\AppData\Local\Temp\ipykernel_19484\640050622.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_rets = (dd_rets.groupby('ticker', group_keys=False)


In [11]:
# Write your code below.
# dd_rets =  dd_rets.repartition(10)
dd_rets_1 =  dd_rets.compute()

In [12]:
dd_rets_1['rolling_avg_return'] = dd_rets_1['returns'].rolling(window = 10).mean()
dd_rets_1

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Adj_Close_lag,returns,hi_lo_range,Close_lag,rolling_avg_return
ticker,,,,,,,,,,,,,,,
RVTY,2019-01-02,77.410004,77.739998,75.830002,76.919998,75.809509,497600,Health Care,Health Care Equipment,2019,NaN,NaN,1.909996,NaN,NaN
RVTY,2019-01-03,76.430000,76.930000,72.160004,72.790001,71.739143,824000,Health Care,Health Care Equipment,2019,75.809509,-0.053692,4.769997,76.919998,NaN
RVTY,2019-01-04,73.919998,76.790001,73.919998,75.889999,74.794388,646200,Health Care,Health Care Equipment,2019,71.739143,0.042588,2.870003,72.790001,NaN
RVTY,2019-01-07,75.889999,77.779999,75.709999,76.720001,75.612419,601800,Health Care,Health Care Equipment,2019,74.794388,0.010937,2.070000,75.889999,NaN
RVTY,2019-01-08,77.830002,78.900002,77.019997,78.769997,77.632812,405300,Health Care,Health Care Equipment,2019,75.612419,0.026720,1.880005,76.720001,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WM,2024-10-22,210.589996,211.649994,209.490005,211.050003,211.050003,896300,Industrials,Environmental & Facilities Services,2024,211.820007,-0.003635,2.159988,211.820007,0.001151
WM,2024-10-23,210.289993,212.380005,209.809998,211.800003,211.800003,1240000,Industrials,Environmental & Facilities Services,2024,211.050003,0.003554,2.570007,211.050003,-0.000195
WM,2024-10-24,210.940002,210.940002,208.449997,208.910004,208.910004,1113000,Industrials,Environmental & Facilities Services,2024,211.800003,-0.013645,2.490005,211.800003,-0.001093


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

- Converting to pandas for calculating the mobving avearage return is common when dataset is smaller. 
- Dask could be better option while working with larger dataset that required distributed computing or when you need more to perform operations that exceed memory capacity. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.